In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import os
import glob
from pprint import pprint
import json
import pandas as pd
import shutil

In [ ]:
if __name__ == "__main__":
    #đường dẫn đến thư mục video và annotation
    #root_path = "/content/drive/MyDrive/YOLO-FOOTBALL/data/football_train"
    root_path = "/content/drive/MyDrive/YOLO-FOOTBALL/data/football_test"
    output_path = "/content/drive/MyDrive/YOLO-FOOTBALL/FootBall_Yolo/Football_Yolo_Dataset"
    #lấy danh sách các đường dẫn video có định dạng .mp4 từ root_path
    video_paths = list(glob.iglob("{}/*/*.mp4".format(root_path)))
    #pprint(video_path)
    #print("\n")
    #lấy danh sách các đường dẫn annotations có định dạng .json từ root_path
    anno_paths = list(glob.iglob('{}/*/*.json'.format(root_path)))
    #lấy phần giao nhau giữa .mp4 và json
    video_wo_ext = [video_path.replace(".mp4","") for video_path in video_paths]
    anno_wo_ext = [anno_paths.replace(".json","") for anno_path in anno_paths]
    #pprint(video_wo_ext)
    #dùng toán tử &
    paths = list(set(video_wo_ext) & set(anno_wo_ext))

    is_train = False
    mode = "train" if is_train else "val"
    #kiểm tra và tạo tư mục cho train hoặc val
    if not os.path.isdir(output_path) and is_train :
        #tạo thư mục images và labels cho train nếu không tồn tại
        os.makedirs(os.path.join(output_path,"image",mode),exit_ok = True)
        os.makedirs(os.path.join(output_path,'image',mode),exit_ok = True)
    else :
        #tạo thư mục images và labels cho cho val
        os.makedirs(os.path.join(output_path ,'image',mode),exit_ok = True)
        os.makedirs(os.path.join(output_path,'image',mode),exit_ok = True)
    #lặp qua từng video có annotation tương ứng
    for idx , path in enumerate(paths):
        video = cv2.VideoCapture('{}.mp4'.format(path)) #mở video bằng OpenCV
        num_frame = video.get(cv2.CAP_PRP_FRAME_COUNT) #lấy ra số frame tương ứng của video
        #print(num_frame)

        #mở file annotation tương ứng dưới dạng Json
        with open("{}.json".format(path)) as json_file:
            json_data = json.load(json_file)
            print(len(json_data['images']))
        #kiểm tra số frame trong video và annotation phải khớp với nhau
        if num_frame != len(json_data['images'])
            print("something is wrong with the game{}".format(path))
            paths.remove(path) #loại bỏ các frame không hợp lệ
        #lấy chiều rộng và chiều cao của frame từ file annotation
        width = json_data['images'][0]['width']
        height = json_data['images'][0]['height']

        #lấy tất cả đối tượng có category_id là 3 và 4 trong annotation
        all_objects = [{'image_id' : obj['image_id'],'bbox' : obj['bbox'] , 'category_id' : obj['category_id']}
                       for obj in json_data['annotations'] if obj['category_id'] in [3,4]]
        #đếm số frame của video
        frame_counter = 0

        #đọc từng frame của video
        with video.isOpened():
            #đọc frame từ video
            flag , frame = video.read()
            if not flag:
                break #thoát khỏi vòng lặp nếu không đọc được frame nào
            #lấy các đối tượng có trong frame hiện tại
            current_object = [obj for obj in all_objects if obj['images_id'] - 1 == frame_counter]
            #pprint(current_object)

            #lưu các frame đọc được vào output
            cv2.imwrite(os.path.join(output_path ,'images',mode,'{}_{}.jpg'.format(idx ,frame_counter)),frame)
            #ghi nhãn các đối tượng vào file .txt cho frame hiện tại
            with open(os.path.join(output-path,'labels',mode,"{}_{}.txt".format(idx ,frame_counter)),'w') as f:
                for obj in current_object:
                    xmin,ymin,w,h = obj['bbox']
                    #chuẩn hóa tọa độ bbox
                    xmin /= width
                    ymin /= height
                    w  /= width
                    h /= width
                    #gán nhãn đối tượng dựa vào category_id : 0 cho category_id = 4 , 1 cho category_id = 3
                    if obj['category_id'] = 4:
                        category = 0
                    else :
                        category = 1
                    #ghi nhãn với định dạng YOLO
                    f.write("{} {:05f} {:05f} {:05f} {:05f}".format(category,xmin+w/2,ymin+h/2,w,h))
            frame_counter +=1

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.26 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 35.7/107.7 GB disk)


In [ ]:
# Bắt đầu huấn luyện
!python /content/drive/MyDrive/YOLO-FOOTBALL/yolov5-master/train.py --img 640 --batch 16 --epochs 10 --data /content/drive/MyDrive/YOLO-FOOTBALL/footbal.yaml \
--weights /content/drive/MyDrive/YOLO-FOOTBALL/yolov5n.pt --project '/content/drive/MyDrive/YOLO-FOOTBALL/results' --name 'detection_result' --rect


2024-10-31 19:11:23.658781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 19:11:23.683669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 19:11:23.691765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/drive/MyDrive/YOLO-FOOTBALL/yolov5n.pt, cfg=, data=/content/drive/MyDrive/YOLO-FOOTBALL/footbal.yaml, hyp=drive/MyDrive/YOLO-FOOTBALL/yolov5-master/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=drive/

In [ ]:
!python /content/drive/MyDrive/YOLO-FOOTBALL/yolov5-master/detect.py \
                --weights /content/drive/MyDrive/YOLO-FOOTBALL/results/detection_result/weights/best.pt \
                --source /content/drive/MyDrive/YOLO-FOOTBALL/data/football_test/Match_1864_1_0_subclip/Match_1864_1_0_subclip.mp4 \
                --imgsz 640 --project '/content/drive/MyDrive/YOLO-FOOTBALL/rusult_test' --name 'test_result_detection'

detect: weights=['/content/drive/MyDrive/YOLO-FOOTBALL/results/detection_result/weights/best.pt'], source=/content/drive/MyDrive/YOLO-FOOTBALL/data/football_test/Match_1864_1_0_subclip/Match_1864_1_0_subclip.mp4, data=drive/MyDrive/YOLO-FOOTBALL/yolov5-master/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/YOLO-FOOTBALL/rusult_test, name=test_result_detection, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2024-10-22 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
video 1/1 (1/1500) /content/drive/MyDrive/YOLO-FOOTBALL/data/football_test/Match_1864_1_0_subclip/Match_1864_1_0_subc

In [ ]:
!python /content/drive/MyDrive/YOLO-FOOTBALL/yolov5-master/detect.py \
                --weights /content/drive/MyDrive/YOLO-FOOTBALL/results/detection_result/weights/best.pt \
                --source /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4 \
                --imgsz 640 --project '/content/drive/MyDrive/YOLO-FOOTBALL/rusult_test' --name 'test_result_detection'

Streaming output truncated to the last 5000 lines.
video 1/1 (1634/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x640 23 players, 1 ball, 65.7ms
video 1/1 (1635/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x640 25 players, 2 balls, 63.3ms
video 1/1 (1636/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x640 25 players, 2 balls, 56.7ms
video 1/1 (1637/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x640 25 players, 2 balls, 71.0ms
video 1/1 (1638/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x640 25 players, 2 balls, 59.4ms
video 1/1 (1639/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x640 25 players, 3 balls, 61.3ms
video 1/1 (1640/6631) /content/drive/MyDrive/YOLO-FOOTBALL/Match_2031_5_0_test/Match_2031_5_0_test.mp4: 224x64